# Test the pathline code

In [ ]:
import numpy as np
import pylab as plt
import xarray as xr
from pypism.trajectory import compute_trajectory, trajectories_to_geopandas
from pypism.interpolation import velocity_at_point
from shapely import Point
from osgeo import ogr, osr
from joblib import Parallel, delayed
import geopandas as gp
from typing import Union
from tqdm.auto import tqdm
import pandas as pd

In [ ]:
def create_circular() -> xr.Dataset:
    """
    Create xr.Dataset with radial velocity field
    """
    time = pd.date_range("2000-01-01", periods=1)
    reference_time = pd.Timestamp("2000-01-01")

    nx = 20_001
    ny = 20_001
    x = np.linspace(-100e3, 100e3, nx)
    y = np.linspace(-100e3, 100e3, ny)
    X, Y = np.meshgrid(x, y)

    # Directional vectors
    vx = -Y / np.sqrt(X**2 + Y**2) * 250
    vy = X / np.sqrt(X**2 + Y**2) * 250
    v = np.sqrt(vx**2 + vy**2)

    vx = vx.reshape(1, ny, nx)
    vy = vy.reshape(1, ny, nx)
    v = v.reshape(1, ny, nx)

    v_err = v / 10
    vx_err = np.abs(vx / 20)
    vy_err = np.abs(vy / 20)

    coords = {
        "x": (
            ["x"],
            x,
            {
                "units": "m",
                "axis": "X",
                "standard_name": "projection_x_coordinate",
                "long_name": "x-coordinate in projected coordinate system",
            },
        ),
        "y": (
            ["y"],
            y,
            {
                "units": "m",
                "axis": "Y",
                "standard_name": "projection_y_coordinate",
                "long_name": "y-coordinate in projected coordinate system",
            },
        ),
        "time": (["time"], time, {}),
    }

    ds = xr.Dataset(
        {
            "vx": xr.DataArray(
                data=vx,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in x-direction", "units": "m/yr"},
            ),
            "vy": xr.DataArray(
                data=vy,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in y-direction", "units": "m/yr"},
            ),
            "v": xr.DataArray(
                data=v,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={
                    "standard_name": "magnitude",
                    "units": "m/yr",
                    "grid_mapping": "polar_stereographic",
                },
            ),
            "vx_err": xr.DataArray(
                data=vx_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in x-direction", "units": "m/yr"},
            ),
            "vy_err": xr.DataArray(
                data=vy_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in y-direction", "units": "m/yr"},
            ),
            "v_err": xr.DataArray(
                data=v_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={
                    "standard_name": "magnitude",
                    "units": "m/yr",
                    "grid_mapping": "polar_stereographic",
                },
            ),
        },
        attrs={"Conventions": "CF-1.7"},
    )
    ds["Polar_Stereographic"] = int()
    ds.Polar_Stereographic.attrs["grid_mapping_name"] = "polar_stereographic"
    ds.Polar_Stereographic.attrs["false_easting"] = 0.0
    ds.Polar_Stereographic.attrs["false_northing"] = 0.0
    ds.Polar_Stereographic.attrs["latitude_of_projection_origin"] = 90.0
    ds.Polar_Stereographic.attrs["scale_factor_at_projection_origin"] = 1.0
    ds.Polar_Stereographic.attrs["standard_parallel"] = 70.0
    ds.Polar_Stereographic.attrs["straight_vertical_longitude_from_pole"] = -45
    ds.Polar_Stereographic.attrs["proj_params"] = "epsg:3413"
    
    return ds


In [ ]:
ds = create_circular()
Vx = np.squeeze(ds["vx"].to_numpy())
Vy = np.squeeze(ds["vy"].to_numpy())
V = np.squeeze(ds["v"].to_numpy())
x = ds["x"].to_numpy()
y = ds["y"].to_numpy()
nx = len(x)
ny = len(y)

## Create doc-string examples

In [ ]:
import numpy as np
from shapely import Point

nx = 201
ny = 401
x = np.linspace(-100e3, 100e3, nx)
y = np.linspace(-100e3, 100e3, ny)
X, Y = np.meshgrid(x, y)

# Directional vectors
vx = -Y / np.sqrt(X**2 + Y**2) * 250
vy = X / np.sqrt(X**2 + Y**2) * 250

p = Point(0, -50000)

pts, pts_error_estim = compute_trajectory(p, vx, vx, x, y, dt=1, total_time=10)

In [ ]:
dt_trajs[0][0]

In [ ]:
starting_point = Point(0, -1_000)
r = starting_point.distance(Point(0, 0))
circ = 2 * r * np.pi
vx, vy = velocity_at_point(Vx, Vy, x, y, starting_point)
v = np.sqrt(vx**2 + vy**2)
total_time = circ / v    

dts = np.logspace(-3, 2, 6)
dt_trajs = {}

progress = tqdm(dts, total=len(dts), leave=False, position=0)
for dt in progress:
    progress.set_description(f"Time step {dt}")
    pts, pts_error_estim = compute_trajectory(starting_point, Vx, Vy, x, y, total_time=total_time+dt, dt=dt, reverse=True)
    dt_trajs[dt] = pts


Integration trajectory:  83%|█████████████████████████████████████████████████████████████▉             | 20.75400000000238/25.133741228718346 [00:18<00:03,  1.11it/s]
Integration trajectory:  83%|██████████████████████████████████████████████████████████████▎            | 20.86600000000252/25.133741228718346 [00:18<00:03,  1.11it/s]
Integration trajectory:  83%|█████████████████████████████████████████████████████████████▊            | 20.979000000002657/25.133741228718346 [00:19<00:03,  1.12it/s]
Integration trajectory:  84%|██████████████████████████████████████████████████████████████            | 21.092000000002795/25.133741228718346 [00:19<00:03,  1.12it/s]
Integration trajectory:  84%|██████████████████████████████████████████████████████████████▍           | 21.205000000002933/25.133741228718346 [00:19<00:03,  1.12it/s]
Integration trajectory:  85%|███████████████████████████████████████████████████████████████▌           | 21.31800000000307/25.133741228718346 [00:19<00:03,  1.

In [ ]:
fig, ax = plt.subplots(1, 1)
dists =[]
for dt, traj in dt_trajs.items():
    dist = traj[-1].distance(traj[0])
    dists.append(dist)
    ax.plot(dt, dist, ".")
    ax.set_xscale("log")
    ax.set_yscale("log")
dists = np.array(dists)

In [ ]:
np.diff(dists) / dists[1:]

In [ ]:
dists[1:]

In [ ]:
dts

In [ ]:
vx

In [ ]:
traj

In [ ]:
def create_linear() -> xr.Dataset:
    """
    Create xr.Dataset with radial velocity field
    """
    time = pd.date_range("2000-01-01", periods=1)
    reference_time = pd.Timestamp("2000-01-01")

    nx = 201
    ny = 201
    x_min = -1
    x_max = 1
    y_min = -1
    y_max = 1
    x = np.linspace(x_min, x_max, nx)
    y = np.linspace(y_min, y_max, ny)
    X, Y = np.meshgrid(x, y)

    # Directional vectors
    vx = X
    vy = -Y
    v = np.sqrt(vx**2 + vy**2)

    vx = vx.reshape(1, ny, nx)
    vy = vy.reshape(1, ny, nx)
    v = v.reshape(1, ny, nx)

    v_err = v / 10
    vx_err = np.abs(vx / 20)
    vy_err = np.abs(vy / 20)

    coords = {
        "x": (
            ["x"],
            x,
            {
                "units": "m",
                "axis": "X",
                "standard_name": "projection_x_coordinate",
                "long_name": "x-coordinate in projected coordinate system",
            },
        ),
        "y": (
            ["y"],
            y,
            {
                "units": "m",
                "axis": "Y",
                "standard_name": "projection_y_coordinate",
                "long_name": "y-coordinate in projected coordinate system",
            },
        ),
        "time": (["time"], time, {}),
    }

    ds = xr.Dataset(
        {
            "vx": xr.DataArray(
                data=vx,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in x-direction", "units": "m/yr"},
            ),
            "vy": xr.DataArray(
                data=vy,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in y-direction", "units": "m/yr"},
            ),
            "v": xr.DataArray(
                data=v,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={
                    "standard_name": "magnitude",
                    "units": "m/yr",
                    "grid_mapping": "polar_stereographic",
                },
            ),
            "vx_err": xr.DataArray(
                data=vx_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in x-direction", "units": "m/yr"},
            ),
            "vy_err": xr.DataArray(
                data=vy_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={"standard_name": "velocity in y-direction", "units": "m/yr"},
            ),
            "v_err": xr.DataArray(
                data=v_err,
                dims=["time", "y", "x"],
                coords=coords,
                attrs={
                    "standard_name": "magnitude",
                    "units": "m/yr",
                    "grid_mapping": "polar_stereographic",
                },
            ),
        },
        attrs={"Conventions": "CF-1.7"},
    )
    ds["Polar_Stereographic"] = int()
    ds.Polar_Stereographic.attrs["grid_mapping_name"] = "polar_stereographic"
    ds.Polar_Stereographic.attrs["false_easting"] = 0.0
    ds.Polar_Stereographic.attrs["false_northing"] = 0.0
    ds.Polar_Stereographic.attrs["latitude_of_projection_origin"] = 90.0
    ds.Polar_Stereographic.attrs["scale_factor_at_projection_origin"] = 1.0
    ds.Polar_Stereographic.attrs["standard_parallel"] = 70.0
    ds.Polar_Stereographic.attrs["straight_vertical_longitude_from_pole"] = -45
    ds.Polar_Stereographic.attrs["proj_params"] = "epsg:3413"
    
    return ds


In [ ]:
ds = create_linear()

$$\frac{\mathrm{d}\mathbf{r}}{\mathrm{d}t} = \mathbf{u} = x\mathbf{i}  - y \mathbf{j} $$

$$\mathbf{r}(t) = \mathbf{r}_0 + \int_{0}^{t'} \mathbf{u}(t') \mathrm{d}t'$$

$$\mathbf{r}(t) = \mathbf{r}(t_0) \left(e^{t}\mathbf{i} + e^{-t}\mathbf{j}\right)$$

In [ ]:
Vx = np.squeeze(ds["vx"].to_numpy())
Vy = np.squeeze(ds["vy"].to_numpy())
V = np.squeeze(ds["v"].to_numpy())
x = ds["x"].to_numpy()
y = ds["y"].to_numpy()
nx = len(x)
ny = len(y)
total_time = np.exp(1)
starting_point = Point(0.05, 0.95)
pts, pts_error_estim = compute_trajectory(starting_point, Vx, Vy, x, y, total_time=total_time, dt=dt)

In [ ]:
def exact_solution(x0, t):
    x = x0.x * np.exp(t)
    y = x0.y * np.exp(-t)
    return Point(x, y)

In [ ]:
r_exact = exact_solution(starting_point, total_time)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(12, 12))
ds["v"].plot(ax=ax)
ax.scatter([p.x for p in pts], [p.y for p in pts], c="k", s=2)
ax.scatter(r_exact.x, r_exact.y, c="r")

In [ ]:
diffs = []
for dt in [1, 0.1, 0.001]:
    pts, pts_error_estim = compute_trajectory(starting_point, Vx, Vy, x, y, total_time=total_time, dt=dt)
    diffs.append(r_exact.distance(pts[-1]))

In [ ]:
plt.scatter([1, 0.1, 0.001], diffs)